## Setup

If you haven't already, install the toolkit and dependencies using the [Setup](./00-Setup.ipynb) notebook.

## Introduction

The [Model Context Protocol](https://modelcontextprotocol.io/introduction) (MCP) is an open protocol that standardizes how applications provide context to LLMs.

In this example, we're going to create a 'catalog' of tools, one per tenant in a [multi-tenant](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/lexical-graph/multi-tenancy.md) graph. Each tool is capable of answering domain-specific questions based on the data in its tenant graph. This catalog will be advertised to clients via an MCP server. Clients (typically agents and LLMs) can then browse the catalog and choose appropriate tools for addressing their information goals.

Each tool in the catalog is accompanied by an auto-generated description that helps a client understand the domain, scope, potential uses and kinds of questions covered by the tool. The catalog also includes a 'search' tool, which, given the name of an entity or concept, recommends one or more domain tools with knowledge of the search term.

### Additional setup

#### Create tenant-specific lexical graphs

Besides the initial setup described above, this example requires you to have created two tenant-specific lexical graphs: a Neptune documentation lexical graph (installed in the _default_ tenant graph), and an Amazon OpenSearch documentation lexical graph (installed in the `aoss` tenant graph):

  - To create the Neptune documentation lexical graph, run either notebook [01-Combined-Extract-and-Build](./01-Combined-Extract-and-Build.ipynb) or notebook [02-Separate-Extract-and-Build](./02-Separate-Extract-and-Build.ipynb).
  - To create the Amazon OpenSearch documentation lexical graph, run the create cell in notebook [05-Multi-Tenancy](./05-Multi-Tenancy.ipynb)

#### Provide Bedrock model access for Claude 3.7 Sonnet

Ensufre you have provided model acess in Amazon Bedrock for `anthropic.claude-3-7-sonnet-20250219-v1:0`

#### Install additional dependencies

The last thing you need to do is install these additional dependencies:

In [1]:
!pip install fastmcp strands-agents

## Create an MCP server

The following cell creates an MCP server that hosts a catalog of tools – one per tenant graph. The cell takes a few seconds to run while the tool descriptions are auto-generated.

In [2]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory, VectorStoreFactory
from graphrag_toolkit.lexical_graph.protocols import create_mcp_server

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

mcp_server = create_mcp_server(graph_store, vector_store)

print('Server initialized')

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Server initialized


### Start the server

The cell below starts the MCP server using the Streamable HTTP transport on a background thread.

In [3]:
import threading

def run_server():
    mcp_server.run(transport='streamable-http', log_level='warning')
    
thread = threading.Thread(target=run_server)
thread.start()

[05/28/25 14:26:20] INFO     Starting MCP server 'LexicalGraphServer' with transport                  ]8;id=995065;file:///home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=333750;file:///home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/fastmcp/server/server.py#823\823]8;;\
                             'streamable-http' on http://127.0.0.1:8000/mcp                                        

INFO:FastMCP.fastmcp.server.server:Starting MCP server 'LexicalGraphServer' with transport 'streamable-http' on http://127.0.0.1:8000/mcp


## Create an MCP client and AI agent

[Strands Agents](https://strandsagents.com/latest/) is an open source SDK that takes a model-driven approach to building and running AI agents in just a few lines of code.

In the cell below we create an MCP client that we can then use in a Strands Agent for answering cross-domain questions.

In [4]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

def create_streamable_http_transport():
    return streamablehttp_client('http://localhost:8000/mcp/')

mcp_client = MCPClient(create_streamable_http_transport)

### Inspect the tool descriptions

The code below prints out the tool descriptions that have been auto-generated from the tenant graphs. Each tool is named after its tenant. The tool for the default tenant graph is named `tenant_`. Note that besides the tenant-specific tools, there is also a `search_` tool.

In [5]:
from strands import Agent

with mcp_client:
    
    tools = mcp_client.list_tools_sync()
    
    for tool in tools:
        print(f"{tool.tool_spec['name']}: {tool.tool_spec['description']}")
        print('\n-------------------------------------\n')

default_: Domain: Amazon Neptune and Graph Database Technologies

Scope: This knowledge base covers Amazon Neptune, a fully managed graph database service, along with related graph database technologies, query languages, and AWS services. It encompasses software components, documentation, and roles involved in the Neptune ecosystem.

Uses: This knowledge base can be utilized for understanding Amazon Neptune's capabilities, its integration with other AWS services, supported query languages, and how it compares to other graph database solutions. It's valuable for developers, database administrators, and cloud architects working with graph databases, particularly in AWS environments. The knowledge base can assist in decision-making for database selection, understanding data loading processes, and identifying compatible tools and services for Neptune.

Example questions:
1. What query languages does Amazon Neptune support?
2. How does Neptune integrate with other AWS services?
3. What are 

### Create an agent and ask a question

We can now create a Strands AI Agent, and ask a question. The agent will choose the most appropriate tools for answering the question.

In [7]:
with mcp_client:

    tools = mcp_client.list_tools_sync()
    agent = Agent(tools=tools)
    
    response = agent("Tell me about Amazon Neptune and recent legal battles in which Amazon has been involved")

I'll help you with information about Amazon Neptune and recent legal battles involving Amazon. Let me break this down into two parts using the appropriate tools.

First, let me get information about Amazon Neptune:
Tool #3: default_
Now, let me look for information about Amazon's recent legal battles. Since the Neptune knowledge base is focused on the database technology and not Amazon's legal issues, I'll need to search for an appropriate tool first:
Tool #4: search_

Tool #5: multihop
Based on the information I've gathered, I can now provide you with details about Amazon Neptune and recent legal battles involving Amazon:

## Amazon Neptune

Amazon Neptune is a fully managed graph database service that makes it easy to work with graph data in the AWS Cloud. Here are its key features and capabilities:

1. **Graph Database Types**:
   - Supports property graphs with Gremlin and openCypher query languages
   - Supports RDF (Resource Description Format) graphs with SPARQL query language

